# 分类学习
## 决策树 Decision Tree

In [16]:
# 绘制Mermaid图表的工具，来自 https://gist.github.com/MLKrisJohnson/2d2df47879ee6afd3be9d6788241fe99
import base64
from IPython.display import Image, display

def mm_ink(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  return "https://mermaid.ink/img/" + base64_string

def mm_display(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, display it."""
  display(Image(url=mm_ink(graphbytes)))

def mm(graph):
  """Given a string containing a Mermaid-format graph, display it."""
  graphbytes = graph.encode("ascii")
  mm_display(graphbytes)

def mm_link(graph):
  """Given a string containing a Mermaid-format graph, return URL for display."""
  graphbytes = graph.encode("ascii")
  return mm_ink(graphbytes)
  
def mm_path(path):
  """Given a path to a file containing a Mermaid-format graph, display it"""
  with open(path, 'rb') as f:
    graphbytes = f.read()
  mm_display(graphbytes)

### 概要
决策树是主要用于分类和回归任务的数据结构，它由节点和连接这两部分组成。
 - 节点：每个节点都代表了一个数据标签
 - 连接：由父节点连接到子节点的标签特征。
我们下面会用一个简单的样例来解释决策树。

首先，我们需要一些数据，这些数据应该具备自己的数据标签和观察到的特征。这里我们以“学生是否通过考试”为案例，示例如下：
| 学习时长(StudyDuration:H/M/L) | 是否复习(Reviewed:Y/N) | 睡眠质量(S(leep)Q(uality):G/A(verage)/P(oor)) | 是否通过考试(Pass/Fail) |
|--------|--------|--------|-----------|
| 高     | 是    | 好     | 是        |
| 高     | 否    | 好     | 否        |
| 中     | 是    | 一般   | 是        |
| 低     | 否    | 差     | 否        |
| 低     | 是    | 好     | 否        |
| 低     | 是    | 一般   | 是        |
| 中     | 否    | 差     | 否        |
| 高     | 是    | 一般   | 是        |

随后，我们需要选取**特征**和**目标变量**。
 - 特征：特征指的是我们用来进行判断决策的指标，比如上表中的学习时长，是否复习等
 - 目标变量：目标变量指的是我们需要预测的指标，只能有一个，就是上表里的是否通过考试
接下来我们需要对这张数据表进行观察并构建一棵决策树。构建决策树时我们需要使用一些算法来确定应该以什么特征进行判断决策，但在这里先暂时忽略，就以最基础的直觉进行构建，如下：

In [28]:
with open('分类学习/决策树1.data', 'r', encoding='utf-8') as file:
    mm(file.read())

在这里，我们展示了一个纯粹直觉构建的决策树图表。当我们拿到一个新的数据，比如下表时，就可以根据上面构建的决策树进行预测。
| 学习时长 | 是否复习 | 睡眠质量 | 是否通过考试 |
|--------|--------|--------|-----------|
| 低     | 否    | 好     | ？        |

我们根据新的数据和决策树，首先判断学习时长，于是走L路线，然后判断是否复习，走H路线，随后判断睡眠质量，走G路线，最后得到的结果就直接是G边对应的叶子节点，也即Fail。于是我们就可以预测在这种情况下这位考生不能通过考试。

In [29]:
with open('分类学习/决策树2.data', 'r', encoding='utf-8') as file:
    mm(file.read())

## ID3算法（第三代迭代二叉树算法）
### 概述
我们在实际的模型学习里当然不可能靠人的直觉来判断决策树应该以什么边和节点来构建，所以我们必须要有一个算法来代替直觉找到“最符合数据分布”的决策树。在这里，我们会使用ID3算法来进行处理。

迭代二叉树算法的计算原理是对整个数据集进行“分裂”，也就是基于标签能提供的信息增益对整个数据集进行拆分。在选择出信息增益最大的标签之后，就会将数据集按这个标签进行分割，随后就像二叉树算法一样，对分割后的每一个子数据集重新执行这个算法，一直执行到所有标签全部被用来分割过一遍，或者达到预设的其他终止条件为止。

### 熵 Entropy
在上文中我们提到了**信息增益**这个关键词，在ID3算法里，信息增益指的是通过数据集的信息熵获得的。

从直观上理解，信息熵可以这么解释：当一个数据集里元素分布得越均匀时，这个数据集的熵就越高。反之，如果这个数据集里元素分布得不均匀，那么这个数据集的熵就越低。极端情况是，如果一个数据集中所有元素都分布在同一个类别，那么这个数据集的熵就是0，也就是熵此时最低。

通过对数据集中指定标签列进行分析，我们可以得到针对当前标签的数据集熵，计算公式如下：
$$H(s)=-\sum^{n}_{i=1}{\left( p_i\log_2{p_i} \right)}$$
其中：
 - $S$是数据集，$H(S)$是这个数据集的信息熵
 - $n$是标签中可取值的数量，比如上面睡眠质量有3种可能性，那么$n=3$，而是否复习有两种可能性，则$n=2$
 - $p_i$是数据集中第$i$类样本出现的概率
   - 这里的概率实际上就只是出现频率，也就是当前分类出现的次数除以数据集大小，公式为$p_i=\frac{|S_i|}{|S|}$
 - $\log_{2}{p_i}$是样本概率的信息量，选择以2为底是因为信息论里我们使用bit作为信息单位，而bit能表示的组合有2种

### 信息增益 Information Gain
光知道熵还不够，为了判断哪个标签可以提供“最多的信息”或者说哪个标签对结果影响最大，我们还需要知道“如果以这个标签对数据集进行分割，我们可以多知道多少信息”。这本质上也是将人的直觉进行量化，例如，当我们在屋内想知道今天是否下雨的时候，我们首先会看看天气预报，然后才会考虑拉开窗帘看看外面是不是真的下雨了。对决策树来说，这个过程是类似的，我们需要优先找出能提供最多信息的标签来对数据集进行分割

当我们知道数据集的时候，也就可以对信息增益做出计算，公式如下：
$$IG(A,S)=H(S)-\sum_{t \in T}{\left( \frac{|S_t|}{S}H(S_t) \right)}$$
其中:
 - $IG(A,S)$是在给定标签$A$下数据集$S$的信息增益，例如，当我们分析上面是否睡眠这个标签时，$A$就是是否睡眠这个标签，而$S$是整体数据集。当我们计算$IG(A,S)$时，我们本质上是在计算给定标签$A$可以给整体数据集$S$提供多少额外信息
 - $H(S)$是上文提到过的信息熵
 - $T$是所有可能的标签集合，例如，前文数据集的$T$实际上就是{学习时长, 是否复习, 睡眠质量}，而$t$则是$T$下的每一个可能取值
 - $S_t$指的是在我们选择$t$来划分时数据集$S$的信息增益
 - $\frac{S_t}{S}$指的是当前分割后的子数据集占数据集整体的比例，我们需要这个值来对信息熵进行加权平均
这整个公式实际含义是对按标签$A$对数据集$S$分割之后，每一个子数据集$S_t$的信息熵的总和相比$S$来说降低了多少，换言之就是量化这次分割可以给我们提供的额外信息。这一点很重要，因为它可以给我们提供一个量化的参考来选取最重要的标签进行分割。

### 算法伪代码
在我们知道信息熵和信息增益的计算方式后，就可以实际开始执行ID3算法了。算法的伪代码如下：
```python
ID3(S, Attr):
    如果实例具有相同的标签：
        返回一个单一节点的树，该节点的标签为这个共同的类标签
    否则：
        1. 创建一个根节点R
        2. 使用信息增益确定增益最大的最佳分类属性A
        3. 对属性A的每个可能值v：
            a. 在树中为属性A添加一个对应其取值v的新的分支
            b. 定义S_v为S中所有在属性A上取值为v的子集
            c. 如果S_v为空：
                在这个新分支下添加一个叶子节点，标记为S中最常见的标签，这样可以应对训练集中可能的缺省值
            否则：
                在这个新分支下递归调用ID3(S_v, Attr-A)
        4. 返回根节点R
```